In [1]:
# coding: utf-8
import sys, os
sys.path.append(os.path.abspath('~/deep/deep-learning-from-scratch/')) 
import numpy as np
from optimizer import *

class Trainer:

    def __init__(self, network, x_train, t_train, x_test, t_test,
                 epochs=5, mini_batch_size=100,
                 optimizer='SGD', optimizer_param={'lr':0.01}, 
                 evaluate_sample_num_per_epoch=None, verbose=True):
        self.network = network
        self.verbose = verbose
        self.x_train = x_train
        self.t_train = t_train
        self.x_test = x_test
        self.t_test = t_test
        self.epochs = epochs
        self.batch_size = mini_batch_size
        self.evaluate_sample_num_per_epoch = evaluate_sample_num_per_epoch

        # optimizer
        optimizer_class_dict = {'sgd':SGD, 'momentum':Momentum, 'nesterov':Nesterov,
                                'adagrad':AdaGrad, 'rmsprop':RMSprop, 'adam':Adam}
        self.optimizer = optimizer_class_dict[optimizer.lower()](**optimizer_param)
        
        self.train_size = x_train.shape[0]
        self.iter_per_epoch = max(self.train_size / mini_batch_size, 1)
        self.max_iter = int(epochs * self.iter_per_epoch)
        self.current_iter = 0
        self.current_epoch = 0
        
        self.train_loss_list = []
        self.train_acc_list = []
        self.test_acc_list = []

    def train_step(self):
        batch_mask = np.random.choice(self.train_size, self.batch_size)
        x_batch = self.x_train[batch_mask]
        t_batch = self.t_train[batch_mask]
        
        print(x_batch.shape, tbatch.shape)
        
        grads = self.network.gradient(x_batch, t_batch)
        self.optimizer.update(self.network.params, grads)
        
        loss = self.network.loss(x_batch, t_batch)
        self.train_loss_list.append(loss)
        if self.verbose: print("train loss:" + str(loss))
        
        if self.current_iter % self.iter_per_epoch == 0:
            self.current_epoch += 1
            
            x_train_sample, t_train_sample = self.x_train, self.t_train
            x_test_sample, t_test_sample = self.x_test, self.t_test
            if not self.evaluate_sample_num_per_epoch is None:
                t = self.evaluate_sample_num_per_epoch
                x_train_sample, t_train_sample = self.x_train[:t], self.t_train[:t]
                x_test_sample, t_test_sample = self.x_test[:t], self.t_test[:t]
                
            train_acc = self.network.accuracy(x_train_sample, t_train_sample)
            test_acc = self.network.accuracy(x_test_sample, t_test_sample)
            self.train_acc_list.append(train_acc)
            self.test_acc_list.append(test_acc)

            if self.verbose: print("=== epoch:" + str(self.current_epoch) + ", train acc:" + str(train_acc) + ", test acc:" + str(test_acc) + " ===")
        self.current_iter += 1

    def train(self):
        for i in range(self.max_iter):
            self.train_step()

        test_acc = self.network.accuracy(self.x_test, self.t_test)

        if self.verbose:
            print("=============== Final Test Accuracy ===============")
            print("test acc:" + str(test_acc))



In [3]:
import numpy as np

In [16]:
input_dim=(1,80,80)
conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1}
weight_init_std=0.01

filter_num = conv_param['filter_num']
filter_size = conv_param['filter_size']
filter_pad = conv_param['pad']
filter_stride = conv_param['stride']

params = {}
params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)

W = params['W1']

FN, C, FH, FW = W.shape
print(FN, C, FH, FW, W.shape)

#col_W = W.shape(FN, -1).T


30 1 5 5 (30, 1, 5, 5)


In [25]:
col_W = W.reshape(FN,-1)

In [27]:
print(col_W)

[[-1.26843254e-02 -1.06442264e-02 -1.22592687e-02  8.71909666e-03
  -7.49774010e-04  6.06023964e-03  1.44899175e-03 -1.37140882e-02
   1.45838168e-02 -5.26603116e-03 -8.52687665e-04  4.05683494e-03
  -1.77408613e-02  8.38491123e-03  4.50512662e-03  1.19320231e-02
   4.47644232e-03 -1.33345865e-02  2.54798537e-03  8.99860791e-03
  -8.25151939e-05 -9.09624616e-04 -1.08311169e-02  1.12748284e-03
   7.90210098e-03]
 [ 4.62421386e-03  7.05116452e-03  4.80145349e-03  9.60275081e-03
  -2.70434374e-02  8.69243382e-03 -9.64264546e-03 -7.13057306e-03
   1.29997061e-02 -3.09598365e-03  5.28638695e-04 -1.43191994e-03
  -1.71297032e-02  4.43016049e-03  1.70945766e-02 -8.16107930e-04
   3.68819102e-03 -1.66825093e-02 -4.27350217e-03  2.13699473e-02
   2.47752094e-02 -2.79390279e-03 -1.71860076e-02 -1.78224353e-02
   9.15635207e-03]
 [ 7.26015169e-03  4.57514485e-03 -5.75839021e-03 -8.22400840e-03
  -1.61573633e-02  1.62823209e-03  1.25556305e-02 -2.62603259e-03
  -1.55984467e-02 -3.90644573e-03 -1.3